# Costa Rican Household Poverty Level Prediction Tutorial (KOR)

## Library Import and Seaborn Setting

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
%matplotlib inline

sns.set(style='white', context='notebook', palette='deep')

mycols = ["#66c2ff", "#5cd6d6", "#00cc99", "#85e085", "#ffd966", "#ffb366", "#ffb3b3", "#dab3ff", "#c2c2d6"]
sns.set_palette(palette = mycols, n_colors = 4)

## Dataset Input

In [ ]:
train_set = pd.read_csv('../input/costa-rican-household-poverty-prediction/train.csv')
test_set  = pd.read_csv('../input/costa-rican-household-poverty-prediction/test.csv')

print(f'train set has {train_set.shape[0]} rows, and {train_set.shape[1]} features')
print(f'test set has {test_set.shape[0]} rows, and {test_set.shape[1]} features')

In [ ]:
#train_set의 'Target' feature의 분포도 확인 (1을 기준으로 한 비율) (Target 값은 1,2,3,4 총 4개)

target = train_set['Target']
target.value_counts(normalize=True)

## Outlier 처리

In [ ]:
#test_set을 보면 rez_esc가 99.0인 outlier가 존재한다.
#rez_esc는 7세에서 19세 사이의 인구 중 입학에 늦는 연수를 의미한다. (8살에 입학해야했는데 11살에 입학했다면 rez_esc=3)
#https://www.kaggle.com/c/costa-rican-household-poverty-prediction/discussion/61403 Discussion을 참고하면, rez_esc의 최대치는 5로 제한된다고 나와있다.
#따라서, 99.0으로 기록된 rez_esc는 허용되는 최대치인 5로 변환한다.

test_set.loc[test_set['rez_esc'] == 99.0, 'rez_esc'] = 5

## Missing Value 탐색

In [ ]:
#train_data에 대하여...

data_na = train_set.isnull().sum().values / train_set.shape[0] * 100
df_na = pd.DataFrame(data_na, index=train_set.columns, columns=['Count'])
df_na = df_na.sort_values(by=['Count'], ascending=False)

missing_value_count = df_na[df_na['Count'] > 0].shape[0]

print(f'We got {missing_value_count} rows which have missing value in train set ')
df_na.head(6)

#rez_esc: years behind in school
#meaneduc: average years of education for adults(18+)
#v18q1: depends on v18q(owns a tablet)
#v2a1: depends on tipovivi3
#SQBmeaned: meaned squared(???) -> 이 예측에서 필요없는 것으로 판단되므로 이후 step에서 0으로 채울것 (<- 잘 모르겠음...)

In [ ]:
#test_data에 대하여...

data_na = test_set.isnull().sum().values / test_set.shape[0] *100
df_na = pd.DataFrame(data_na, index=test_set.columns, columns=['Count'])
df_na = df_na.sort_values(by=['Count'], ascending=False)

missing_value_count = df_na[df_na['Count']>0].shape[0]

print(f'We got {missing_value_count} rows which have missing value in test set ')
df_na.head(6)

In [ ]:
#Fill NA

def replace_v18q1(x):
    if x['v18q'] == 0:
        return x['v18q']
    else:
        return x['v18q1']

train_set['v18q1'] = train_set.apply(lambda x : replace_v18q1(x),axis=1)
test_set['v18q1'] = test_set.apply(lambda x : replace_v18q1(x),axis=1)

train_set['v2a1'] = train_set['v2a1'].fillna(value=train_set['tipovivi3'])
test_set['v2a1'] = test_set['v2a1'].fillna(value=test_set['tipovivi3'])

## Feature Engineering

In [ ]:
cols = ['edjefe', 'edjefa']
#edjefe: years of education of male head of household
#edjefa: years of education of female head of household

#yes -> 1, no -> 0으로 변환
train_set[cols] = train_set[cols].replace({'no': 0, 'yes':1}).astype(float)
test_set[cols] = test_set[cols].replace({'no': 0, 'yes':1}).astype(float)

In [ ]:
#새로 만드는 feature 4종
train_set['roof_waste_material'] = np.nan
test_set['roof_waste_material'] = np.nan
train_set['electricity_other'] = np.nan
test_set['electricity_other'] = np.nan

#techozinc: =1 지붕의 주요 재료가 금속 포일 또는 징크인 경우
#techoentrepiso: =1 지붕의 주요 재료가 섬유 시멘트, 메자닌인 경우
#techocane: =1 지붕의 주요 재료가 천연 섬유인 경우
#techootro: =1 지붕의 주요 재료가 다른 경우
def fill_roof_exception(x):
    if (x['techozinc'] == 0) and (x['techoentrepiso'] == 0) and (x['techocane'] == 0) and (x['techootro'] == 0):
        return 1
    else:
        return 0

#public: =CNFL, ICE, ESPH/JASEC에서 나오는 전기 1개
#planpri: =민간 발전소에서 나오는 전기 1개
#noelec: =1 주거지에 전기가 없음
#coopele: =협동조합 전기 1개
def fill_no_electricity(x):
    if (x['public'] == 0) and (x['planpri'] == 0) and (x['noelec'] == 0) and (x['coopele'] == 0):
        return 1
    else:
        return 0

#위에서 정의한 함수에 맞춰서 새로운 4종의 feature를 채움.
train_set['roof_waste_material'] = train_set.apply(lambda x : fill_roof_exception(x),axis=1)
test_set['roof_waste_material'] = test_set.apply(lambda x : fill_roof_exception(x),axis=1)
train_set['electricity_other'] = train_set.apply(lambda x : fill_no_electricity(x),axis=1)
test_set['electricity_other'] = test_set.apply(lambda x : fill_no_electricity(x),axis=1)

In [ ]:
#18세 이상이 가장인가? 를 보여주는 feature 생성(head<18>)
def owner_is_adult(x):
    if x['age'] <= 18:
        return 0
    else:
        return 1

train_set['head<18'] = train_set.apply(lambda x : owner_is_adult(x),axis=1)
test_set['head<18'] = test_set.apply(lambda x : owner_is_adult(x),axis=1)

In [ ]:
#더 다양한 feature들 직접생성

#hogar_adul: 어른인 가구원 수
#hogar_mayor: 65세 이상 가구원 수
#hogar_nin: 0~19세인 가구원 수
#hogar_total: 가구원 총 인원수
train_set['adult'] = train_set['hogar_adul'] - train_set['hogar_mayor']
train_set['dependency_count'] = train_set['hogar_nin'] + train_set['hogar_mayor']
train_set['dependency'] = train_set['dependency_count'] / train_set['adult']
train_set['child_percent'] = train_set['hogar_nin']/train_set['hogar_total']
train_set['elder_percent'] = train_set['hogar_mayor']/train_set['hogar_total']
train_set['adult_percent'] = train_set['hogar_adul']/train_set['hogar_total']
test_set['adult'] = test_set['hogar_adul'] - test_set['hogar_mayor']
test_set['dependency_count'] = test_set['hogar_nin'] + test_set['hogar_mayor']
test_set['dependency'] = test_set['dependency_count'] / test_set['adult']
test_set['child_percent'] = test_set['hogar_nin']/test_set['hogar_total']
test_set['elder_percent'] = test_set['hogar_mayor']/test_set['hogar_total']
test_set['adult_percent'] = test_set['hogar_adul']/test_set['hogar_total']

#v2a1: 월세 납부
#hhsize: 가구 쿠기
train_set['rent_per_adult'] = train_set['v2a1']/train_set['hogar_adul']
train_set['rent_per_person'] = train_set['v2a1']/train_set['hhsize']
test_set['rent_per_adult'] = test_set['v2a1']/test_set['hogar_adul']
test_set['rent_per_person'] = test_set['v2a1']/test_set['hhsize']

#hacdor: =1 침실별 과잉 수용
#hacapo: =1 방별 과일 수용
train_set['overcrowding_room_and_bedroom'] = (train_set['hacdor'] + train_set['hacapo'])/2
test_set['overcrowding_room_and_bedroom'] = (test_set['hacdor'] + test_set['hacapo'])/2

#refrig/computer/television: =1 가구에 냉장고/컴퓨터/TV가 있는 경우
train_set['no_appliances'] = train_set['refrig'] + train_set['computer'] + train_set['television']
test_set['no_appliances'] = test_set['refrig'] + test_set['computer'] + test_set['television']

#rh41: 12세 미만의 남성
#r4h2: 12세 이상의 남성
#r4h3: 가구원 남성 합계
#r4m1: 12세 미만의 여성
#r4m2: 12세 이상의 여성
#r4m3: 가구원 여성 합계
#hhsize: 가구원 크기
train_set['r4h1_percent_in_male'] = train_set['r4h1'] / train_set['r4h3']
train_set['r4m1_percent_in_female'] = train_set['r4m1'] / train_set['r4m3']
train_set['r4h1_percent_in_total'] = train_set['r4h1'] / train_set['hhsize']
train_set['r4m1_percent_in_total'] = train_set['r4m1'] / train_set['hhsize']
train_set['r4t1_percent_in_total'] = train_set['r4t1'] / train_set['hhsize']
test_set['r4h1_percent_in_male'] = test_set['r4h1'] / test_set['r4h3']
test_set['r4m1_percent_in_female'] = test_set['r4m1'] / test_set['r4m3']
test_set['r4h1_percent_in_total'] = test_set['r4h1'] / test_set['hhsize']
test_set['r4m1_percent_in_total'] = test_set['r4m1'] / test_set['hhsize']
test_set['r4t1_percent_in_total'] = test_set['r4t1'] / test_set['hhsize']

#bedrooms: 침실 수
train_set['rent_per_room'] = train_set['v2a1']/train_set['rooms']
train_set['bedroom_per_room'] = train_set['bedrooms']/train_set['rooms']
train_set['elder_per_room'] = train_set['hogar_mayor']/train_set['rooms']
train_set['adults_per_room'] = train_set['adult']/train_set['rooms']
train_set['child_per_room'] = train_set['hogar_nin']/train_set['rooms']
train_set['male_per_room'] = train_set['r4h3']/train_set['rooms']
train_set['female_per_room'] = train_set['r4m3']/train_set['rooms']
train_set['room_per_person_household'] = train_set['hhsize']/train_set['rooms']

test_set['rent_per_room'] = test_set['v2a1']/test_set['rooms']
test_set['bedroom_per_room'] = test_set['bedrooms']/test_set['rooms']
test_set['elder_per_room'] = test_set['hogar_mayor']/test_set['rooms']
test_set['adults_per_room'] = test_set['adult']/test_set['rooms']
test_set['child_per_room'] = test_set['hogar_nin']/test_set['rooms']
test_set['male_per_room'] = test_set['r4h3']/test_set['rooms']
test_set['female_per_room'] = test_set['r4m3']/test_set['rooms']
test_set['room_per_person_household'] = test_set['hhsize']/test_set['rooms']

train_set['rent_per_bedroom'] = train_set['v2a1']/train_set['bedrooms']
train_set['edler_per_bedroom'] = train_set['hogar_mayor']/train_set['bedrooms']
train_set['adults_per_bedroom'] = train_set['adult']/train_set['bedrooms']
train_set['child_per_bedroom'] = train_set['hogar_nin']/train_set['bedrooms']
train_set['male_per_bedroom'] = train_set['r4h3']/train_set['bedrooms']
train_set['female_per_bedroom'] = train_set['r4m3']/train_set['bedrooms']
train_set['bedrooms_per_person_household'] = train_set['hhsize']/train_set['bedrooms']

test_set['rent_per_bedroom'] = test_set['v2a1']/test_set['bedrooms']
test_set['edler_per_bedroom'] = test_set['hogar_mayor']/test_set['bedrooms']
test_set['adults_per_bedroom'] = test_set['adult']/test_set['bedrooms']
test_set['child_per_bedroom'] = test_set['hogar_nin']/test_set['bedrooms']
test_set['male_per_bedroom'] = test_set['r4h3']/test_set['bedrooms']
test_set['female_per_bedroom'] = test_set['r4m3']/test_set['bedrooms']
test_set['bedrooms_per_person_household'] = test_set['hhsize']/test_set['bedrooms']

#v18q1: 가구 소유의 태블릿 수
#qmobilephone: 가구 소유의 휴대전화 수
train_set['tablet_per_person_household'] = train_set['v18q1']/train_set['hhsize']
train_set['phone_per_person_household'] = train_set['qmobilephone']/train_set['hhsize']
test_set['tablet_per_person_household'] = test_set['v18q1']/test_set['hhsize']
test_set['phone_per_person_household'] = test_set['qmobilephone']/test_set['hhsize']

#r4t1: 12세 미만인 사람
train_set['age_12_19'] = train_set['hogar_nin'] - train_set['r4t1']
test_set['age_12_19'] = test_set['hogar_nin'] - test_set['r4t1']    

#escolari: 다년간의 교육
train_set['escolari_age'] = train_set['escolari']/train_set['age']
test_set['escolari_age'] = test_set['escolari']/test_set['age']

train_set['rez_esc_escolari'] = train_set['rez_esc']/train_set['escolari']
train_set['rez_esc_r4t1'] = train_set['rez_esc']/train_set['r4t1']
train_set['rez_esc_r4t2'] = train_set['rez_esc']/train_set['r4t2']
train_set['rez_esc_r4t3'] = train_set['rez_esc']/train_set['r4t3']
train_set['rez_esc_age'] = train_set['rez_esc']/train_set['age']
test_set['rez_esc_escolari'] = test_set['rez_esc']/test_set['escolari']
test_set['rez_esc_r4t1'] = test_set['rez_esc']/test_set['r4t1']
test_set['rez_esc_r4t2'] = test_set['rez_esc']/test_set['r4t2']
test_set['rez_esc_r4t3'] = test_set['rez_esc']/test_set['r4t3']
test_set['rez_esc_age'] = test_set['rez_esc']/test_set['age']

In [ ]:
train_set['dependency'] = train_set['dependency'].replace({np.inf: 0})
test_set['dependency'] = test_set['dependency'].replace({np.inf: 0})

print(f'train set has {train_set.shape[0]} rows, and {train_set.shape[1]} features')
print(f'test set has {test_set.shape[0]} rows, and {test_set.shape[1]} features')

In [ ]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

aggr_mean_list = ['rez_esc', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 'parentesco2',
             'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
             'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',]

other_list = ['escolari', 'age', 'escolari_age']

#aggr_mean_list의 feature들에 대해 mean값을 나타내 주는 feature 추가 생성
for item in aggr_mean_list:
    group_train_mean = train_set[item].groupby(train_set['idhogar']).mean()
    group_test_mean = test_set[item].groupby(test_set['idhogar']).mean()
    new_col = item + '_aggr_mean'
    df_train[new_col] = group_train_mean
    df_test[new_col] = group_test_mean

#other_list의 feature에 대해서는 mean, std, min, max, sum값을 나타내주는 추가 feature 생성
for item in other_list:
    for function in ['mean','std','min','max','sum']:
        group_train = train_set[item].groupby(train_set['idhogar']).agg(function)
        group_test = test_set[item].groupby(test_set['idhogar']).agg(function)
        new_col = item + '_' + function
        df_train[new_col] = group_train
        df_test[new_col] = group_test

print(f'new aggregate train set has {df_train.shape[0]} rows, and {df_train.shape[1]} features')
print(f'new aggregate test set has {df_test.shape[0]} rows, and {df_test.shape[1]} features')

In [ ]:
#위의 추가적인 aggregate feature들을 원본 데이터에 추가
df_test = df_test.reset_index()
df_train = df_train.reset_index()

train_agg = pd.merge(train_set, df_train, on='idhogar')
test = pd.merge(test_set, df_test, on='idhogar')

#fill all na as 0
train_agg.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)
print(f'new train set has {train_agg.shape[0]} rows, and {train_agg.shape[1]} features')
print(f'new test set has {test.shape[0]} rows, and {test.shape[1]} features')

In [ ]:
#train_agg 중 parentesco1이 1인값만 train값으로 지정
#parentsco1: 가정의 가장일 경우 값이 1임.
#competition의 데이터 설명에 의하면, 채점 시 가장인 경우의 데이터만 채점하기 때문에 parentsco1값이 1인 row만 사용
#다른 노트북을 보니, 이렇게 하는 사람들이 많았음.
train = train_agg.query('parentesco1==1')

In [ ]:
submission = test[['Id']]

#필요없는 feature들을 지운다. (기준은 notebook에 나와있지 않아 잘 모르겠음. 나름의 feature selection을 한 것으로 보임.)
train.drop(columns=['idhogar','Id', 'tamhog', 'agesq', 'hogar_adul', 'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned'], inplace=True)
test.drop(columns=['idhogar','Id', 'tamhog', 'agesq', 'hogar_adul', 'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned'], inplace=True)

correlation = train.corr()
correlation = correlation['Target'].sort_values(ascending=False)
print(f'The most 20 positive feature: \n{correlation.head(20)}')
print('*'*50)

print(f'The most 20 negative feature: \n{correlation.tail(20)}')

In [ ]:
y = train['Target']

train.drop(columns=['Target'], inplace=True)

#https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro Notebook에서 lightgbm의 parameter를 가져옴.
#위의 노트북에서는 lightgbm을 RandomizedSearchCV를 통해 찾아냈음.
#Parameter는 num_leaves, min_child_samples, subsample, colsample_bytree 4가지

#num_leaves: 하나의 트리가 가질 수 있는 최대 leaf의 개수(높으면 정확도 증가, but 모델 복잡도 증가)
#min_child_samples: 최종 결정 클래스인 leaf node가 되기 위해 최소한으로 필요한 데이터 개체 수
#subsample: 과적합을 제어하기 위해 데이터를 샘플링하는 비율
#colsample_bytree: 개별 트리를 학습할 때마다 무작위로 선택하는 feature의 비율을 제어

#lightgbm에는 이외에도 max_depth, learning_rate, reg_alpha, reg_lambda, n_estimators 등의 parameter들이 있다.
#각각 트리의 최대 깊이, 학습률, L1 규제항, L2 규제항, 반복 수행하는 트리의 개수이다.

clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.1, objective='multiclass',
                             random_state=None, silent=True, metric='None', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.93, min_child_samples = 95, num_leaves = 14, subsample = 0.96)

In [ ]:
#원본 NOTEBOOK에서는 early_stopping을 적용했으나, 적용 시 오류가 발생해 이 notebook에서는 제외함.

#kfold의 개수를 5개로 지정하고, stratified 방식을 적용함.
kfold = 5
kf = StratifiedKFold(n_splits=kfold, shuffle=True)

predicts_result = []
for train_index, test_index in kf.split(train, y):
    print("###")
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)
    predicts_result.append(clf.predict(test))

In [ ]:
#feature importance를 확인하기 위한 코드들

indices = np.argsort(clf.feature_importances_)[::-1]
indices = indices[:75]

# 막대 그래프로 feature importance를 시각화함.
plt.subplots(figsize=(20, 15))
g = sns.barplot(y=train.columns[indices], x = clf.feature_importances_[indices], orient='h', palette = mycols)
g.set_xlabel("Relative importance",fontsize=12)
g.set_ylabel("Features",fontsize=12)
g.tick_params(labelsize=9)
g.set_title("LightGBM feature importance");

In [ ]:
#submission file 제출
submission['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission.to_csv('submission.csv', index = False)

#0.41568이라는 score가 나왔음.